In [1]:
from zipline.pipeline import Pipeline
from zipline.component.research import run_pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

### Combining Filters
Like factors, filters can be combined. Combining filters is done using the `&` (and) and `|` (or) operators. For example, let's say we want to screen for securities that are in the top 10% of average dollar volume and have a latest close price above $20. To start, let's make a high dollar volume filter using an `AverageDollarVolume` factor and `percentile_between`:

In [2]:
dollar_volume = AverageDollarVolume(window_length=30)
high_dollar_volume = dollar_volume.percentile_between(90, 100)

Note: `percentile_between` is a `Factor` method returning a `Filter`.

Next, let's create a `latest_close` factor and define a filter for securities that closed above $20:

In [3]:
latest_close = USEquityPricing.close.latest
above_20 = latest_close > 20

Now we can combine our `high_dollar_volume` filter with our `above_20` filter using the `&` operator:

In [4]:
tradeable_filter = high_dollar_volume & above_20

This filter will evaluate to `True` for securities where both `high_dollar_volume` and `above_20` are `True`. Otherwise, it will evaluate to `False`. A similar computation can be made with the `|` (or) operator.

If we want to use this filter as a screen in our pipeline, we can simply pass `tradeable_filter` as the `screen` argument.

In [5]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)

    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30

    dollar_volume = AverageDollarVolume(window_length=30)
    high_dollar_volume = dollar_volume.percentile_between(90, 100)

    latest_close = USEquityPricing.close.latest
    above_20 = latest_close > 20

    tradeable_filter = high_dollar_volume & above_20

    return Pipeline(
        columns={
            'percent_difference': percent_difference
        },
        screen=tradeable_filter
    )

When we run this, our pipeline output now only includes ~700 securities.

In [6]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
print('Number of securities that passed the filter: %d' % len(result))
result

Number of securities that passed the filter: 135


percent_difference
2015-05-05 00:00:00+00:00 Equity(000009 [中国宝安])             0.100745
                          Equity(000039 [中集集团])             0.075515
                          Equity(000063 [中兴通讯])             0.161306
                          Equity(000333 [美的集团])             0.075422
                          Equity(000338 [潍柴动力])             0.029796
                          Equity(000400 [许继电气])             0.027624
                          Equity(000423 [东阿阿胶])             0.066138
                          Equity(000623 [吉林敖东])             0.027189
                          Equity(000651 [格力电器])             0.118010
                          Equity(000686 [东北证券])             0.068752
                          Equity(000712 [锦龙股份])             0.066393
                          Equity(000728 [国元证券])             0.018223
                          Equity(000758 [中色股份])             0.138105
                          Equity(000768 [中航飞机])             0.113023
                          Equity(000776 [广发证券])             0.009876
                          Equity(000777 [中核科技])             0.099409
                          Equity(000800 [一汽轿车])             0.138927
                          Equity(000831 [五矿稀土])             0.080659
                          Equity(000858 [五 粮 液])            0.080207
                          Equity(000876 [新 希 望])            0.048866
                          Equity(000917 [电广传媒])             0.060927
                          Equity(000977 [浪潮信息])             0.072745
                          Equity(000998 [隆平高科])             0.095887
                          Equity(002008 [大族激光])             0.122694
                          Equity(002030 [达安基因])             0.084572
                          Equity(002065 [东华软件])             0.065347
                          Equity(002146 [荣盛发展])             0.030474
                          Equity(002183 [怡 亚 通])            0.024515
                          Equity(002195 [二三四五])            -0.087810
                          Equity(002202 [金风科技])             0.065183
...                                                              ...
                          Equity(600739 [辽宁成大])             0.022274
                          Equity(600765 [中航重机])             0.102773
                          Equity(600804 [鹏博士])             -0.024412
                          Equity(600832 [东方明珠])             0.224242
                          Equity(600837 [海通证券])             0.083061
                          Equity(600875 [东方电气])             0.130668
                          Equity(600879 [航天电子])             0.055020
                          Equity(600887 [伊利股份])             0.105535
                          Equity(600893 [航发动力])             0.207417
                          Equity(600895 [张江高科])             0.079763
                          Equity(600958 [东方证券])             0.148714
                          Equity(600999 [招商证券])             0.058018
                          Equity(601002 [晋亿实业])             0.061580
                          Equity(601088 [中国神华])             0.085479
                          Equity(601186 [中国铁建])             0.186533
                          Equity(601198 [东兴证券])             0.076503
                          Equity(601299 [中国北车])             0.291167
                          Equity(601318 [中国平安])             0.051284
                          Equity(601336 [新华保险])            -0.006209
                          Equity(601390 [中国中铁])             0.295437
                          Equity(601519 [*ST智慧])            0.051340
                          Equity(601555 [东吴证券])             0.038074
                          Equity(601601 [中国太保])             0.022773
                          Equity(601607 [上海医药])             0.059263
                          Equity(601628 [中国人寿])             0.004843
                          Equity(601688 [华泰证券])             0.021856
                          Equity(601766 [中国中车]) 

In the next lesson, we'll look at masking factors and filters.